In [1]:
import sys
sys.path.append("../backend")

In [2]:
DATASET = "test"
DATASET_FILE = f"../data/generated/{DATASET}.json"
RESULT_FILE = f"../data/generated/results/{DATASET}_saul_reciprocal_cross_encoder.json"
HALLUCINATION_FILE = f"../data/generated/hallucinations/{DATASET}_saul_reciprocal_cross_encoder.json"

print(f'DATASET_FILE: {DATASET_FILE}')
print(f'RESULT_FILE: {RESULT_FILE}')
print(f'HALLUCINATION_FILE: {HALLUCINATION_FILE}')

DATASET_FILE: ../data/generated/test.json
RESULT_FILE: ../data/generated/results/test_saul_reciprocal_cross_encoder.json
HALLUCINATION_FILE: ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json


In [3]:
PROMPT = """
Given the following QUESTION, DOCUMENT and ANSWER you must analyze the provided answer and determine whether it is faithful to the contents of the DOCUMENT. The ANSWER must not offer new information beyond the context provided in the DOCUMENT. The ANSWER also must not contradict information provided in the DOCUMENT. Output your final verdict by strictly following this format: "PASS" if the answer is faithful to the DOCUMENT and "FAIL" if the answer is not faithful to the DOCUMENT. Show your reasoning. Keep the double quotes in your output.

--
QUESTION (THIS DOES NOT COUNT AS BACKGROUND INFORMATION):
{question}

--
DOCUMENT:
{context}

--
ANSWER:
{answer}

--

Your output should be in JSON FORMAT, with the keys "REASONING" and "SCORE":
{{"REASONING": <your reasoning as bullet points>, "SCORE": <your final score>}}
"""

In [4]:
question = """what is the purpose of the delegated act?"""

#answer = """delegated act specifies the disclosure obligations under article 8 of the taxonomy regulation. this disclosure will help investors and the public to understand the companies’ trajectory towards sustainability through the annual publication of their kpis associated with environmentally sustainable economic activities. the delegated act will therefore increase transparency in the market and help prevent greenwashing by informing investors about companies’ environmental performance"""

answer = "this is a trick"

context = """delegated act specifies the disclosure obligations under article 8 of the taxonomy regulation. the rules set out in the delegated act allows companies to translate the technical screening criteria of the climate delegated act (and the future environmental delegated act) into quantitative economic performance indicators - the kpis - which will be publicly disclosed (e.g. the percentage of environmentally sustainable economic activities in a company’s turnover or capital expenditure). this disclosure will help investors and the public to understand the companies’ trajectory towards sustainability through the annual publication of their kpis associated with environmentally sustainable economic activities. the delegated act will therefore increase transparency in the market and help prevent greenwashing by informing investors about companies’ environmental performance. large financial and non-financial undertakings can use the information disclosed to design credible green financial products such as green bonds or investment funds and, through the public disclosures under the delegated act, channel investor demand towards sustainable projects. market actors who are not covered by the nfrd, such as small and medium size enterprises (smes), can report some or all kpis on a voluntary basis. such market transparency through publishing the percentage of their turnover or investments that is aligned with the taxonomy regulation should help companies to raise the financing for sustainable activities. the taxonomy regulation applies to financial market participants that offer financial products, financial and non-financial undertakings within the scope of directive 2014/95/eu (the non-financial reporting directive - ‘nfrd’) 5 . it also applies to member states and the eu in the context of introducing national and eu-level requirements regarding financial market participants or to issuers for the purpose of labelling financial products or corporate bonds that are marketed as environmentally sustainable. the taxonomy regulation identifies environmentally sustainable economic activities based on technical screening criteria set out in the commission’s delegated acts developed under this regulation 6 . the first delegated act concerning the technical screening criteria for economic activities with significant contribution to climate change mitigation and adaptation (the ‘climate delegated act’) was adopted on 4 june 2021 7 . another delegated act concerning the technical screening criteria for the remaining four environmental objectives (‘the environmental delegated act’) will be developed and adopted later. – the regulatory technical standards referred to in articles 8, 9 and 11 11 of the sfdr relating to the details of the content and presentation of the information referred to in articles 5 and 6 of the taxonomy regulation; the revision of the nfrd, and accounting standards provided in regulation (ec) no 1126/ consultations prior to the adoption of the act on 28 july 2020, the commission published an inception impact assessment for a consultation over a period of six weeks 11 . there were 78 respondents in total, of which 9% were business associations, 9% were companies/business organisations, 9% were non-governmental organisations and 4% were eu citizens. the overall feedback received by the commission indicated that the delegated act is considered as a useful initiative that could help allocate capital to environmentally sustainable economic activities. many respondents expressed a particular interest in the level of detail that future disclosures will be required to include. the non-financial undertakings consulted considered that the three elements to be disclosed outlined in the taxonomy regulation are appropriate. financial undertakings highlighted that criteria should be consistent, comparable and publicly available. many respondents noted a need to avoid administrative burdens and costs linked to data collection, especially for smes. many also mentioned the challenge of collecting taxonomy compliance data within companies and groups and assigning this data to specific business lines"""

In [5]:
import json
with open(RESULT_FILE, "r", encoding='utf-8') as f:
    data = json.load(f)

In [6]:
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:8000/v1", api_key="sk-xxx")

results = []
count_pass = 0

# Ensure the directory exists
os.makedirs(os.path.dirname(HALLUCINATION_FILE), exist_ok=True)

for i, item in enumerate(data["data"]):
    id = item["id"]
    question = item["question"]
    context = "\n".join(map(lambda x: x["text"], item["relevant_embeddings"]))
    answer = item["pred_answer"]
    response = client.chat.completions.create(
        model="lynx",
        messages=[
            {
                "role": "user",
                "content": PROMPT.format(question=question, context=context, answer=answer),
            }
        ],
        temperature=0.0001,
        max_tokens=600
    )
    result = response.choices[0].message.content
    result = result[result.find("{") : result.find("}")+1]
    result = result.replace("'", "\"")
    score = result[-5:-1]
    results.append({
        "id": str(id),
        "result": str(result),
        "score": str(score)
    })
    
    if score == "PASS":
        count_pass += 1
    
    # append result and score to a json file
    with open(HALLUCINATION_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"saved results for {id} in {HALLUCINATION_FILE}")
        print(f"PASS: {count_pass}/{i+1}")

saved results for 390bed57-a631-4986-a794-902e948bf685 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 1/1
saved results for daa6fcab-6f6c-441f-b704-61d18c237ae1 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 2/2
saved results for 52b903bb-d3d3-47c4-a845-9d460dd9a015 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 3/3
saved results for bbb365cf-3b71-4aab-b3c5-b5032751ad58 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 4/4
saved results for fc9b28bc-6d9d-45d5-bc6f-c17515f05b9f in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 5/5
saved results for 79da3883-4ef8-4b98-9c08-7b4e93835a67 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 6/6
saved results for 4701665d-a07f-4ed2-8690-8d066b252c09 in ../data/generated/hallucinations/test_saul_reciprocal_cross_encoder.json
PASS: 7/7
saved results

In [44]:
result = response.choices[0].message.content